# Week 4 - Portfolio Optimization: From Markowitz to Robust Convex Formulations

This notebook covers the theory and background. It assumes a decent knowledge of lin alg, stats, calculus (prerequisites for convex optimization), but if you were able to interpret the previous weeks, this shouldn't present a challenge.

## 0. What problem are we solving?

In the previous weeks, we focused on **predicting asset returns** using historical data and simple machine-learning models.
In this notebook, we address the *next* question:

> **Given predicted returns, how should we allocate capital?**

---

## 1. Portfolio representation - Basics

### 1.1 Assets and weights

We consider a universe containing $n$ financial assets (assume stocks/equities for the rest of this notebook).
A portfolio is described by a **weight vector**
$$
w = (w_1,\dots,w_n) \in \mathbb{R}^n,
$$

where $w_i$ denotes the **fraction of total portfolio value $V$** invested in asset $i$. Additionally let $c$ represent the fraction of portfolio value held as cash in hand.

We impose the normalization constraint
$$
\mathbf{1}^\top w + c = 1,
$$
which simply means that *all capital (including cash) is allocated somewhere*.

> Example: Say you start off with a total wealth of $\$500$. You now buy 2 shares of stock A valued at $\$100$ (i.e. a long position) and borrow and sell 1 share of stock B valued at $\$50$ (you now owe the lender one unit of stock whose value you hold as cash right now, i.e. a short position).
>
> Now, $V=\text{long\_positions - short\_positions + cash} = 2*100 - 1*50 + (500-2*100+50)=500$ (of course), where the cash held is $\$350$.


**Fully invested, long-only.** One very useful and practical constraint (which we will consider hereon) is the *fully invested* and *long-only portfolio* constraint. That is $c=0$ and $w_i>=0 \forall i$.
We write it succinctly as $\mathbf{1}^\top \mathbf{w}=1 ; \mathbf{w} \ge \mathbf{0}$.

### 1.2 Returns and risk

Let the (random) asset returns at time $t$ be represented by a vector
$$
R_t = R \in \mathbb{R}^n.
$$
*Remark: We implicitly make a "time-independence" assumption here, that is, the method of optimization we consider in this project will not make use of the historical $t-1$ units of data and rather treat the "predicted" returns at tick $t$ as the sole source of information, and hence our problems are framed in a time-independent sense.*

From previous weeks, we have estimates of:

* expected returns: $\mu = \mathbb{E}[R]$,
* covariance matrix: $\Sigma = \mathrm{Cov}(R)$.

The portfolio return is
$$
R_p = w^\top R.
$$

By linearity,
$$
\mathbb{E}[R_p] = \mu^\top w,
\qquad
\mathrm{Var}(R_p) = w^\top \Sigma w.
$$

These two expressions — expected return and variance — are the entire mathematical foundation of classical portfolio theory.

---


## 2. Baseline portfolios (why optimization matters)

Before looking at modern portfolio theory, we present some reasonable baseline/heuristic portfolios whose significance is far from just theoretical.

### 2.1 Equal-weight portfolio

The simplest portfolio is
$$
w_i = \frac{1}{n}.
$$

This portfolio:

* ignores all estimates,
* enforces diversification,
* is often a surprisingly strong baseline.

It serves as a **control** against which optimized portfolios should be compared. It is well-studied and often a [suprisingly good baseline](https://palomar.home.ece.ust.hk/papers/2020/ZhouPalomar-TSP2020%20-%20quintile%20portfolio.pdf).

### 2.2 Return-only optimization (naive and impractical)

As a purely hypothetical problem, consider the optimization problem
$$
\max_w \ \mu^\top w
\quad
\text{s.t. } \mathbf{1}^\top w = 1,\ w \ge 0.
$$

The solution is trivial: place **all weight on the single asset with highest predicted return**.

Clearly, this has various flaws:

* extremely sensitive to noise in $\mu$,
* completely undiversified,
* unrealistic in practice.

Hence modern portfolio theory focusses heavily on risk control.
> Remark: In many cases, the lack of accuracy in "predicted expected returns $\hat\mu$" combined with the high sensitivity lead to a category of portfolios where expected returns are not used at all. In fact this will be the motivation for the robust optimization methods we discuss later.

### Reference
Chapter 6 from the portfolio optimization book. If you're confused about the definitions the introductory chapters in the book (5, 6) cover it well; the README is intentionally concise.

---


## 3. Markowitz’s mean–variance idea

Harry Markowitz ([1952](https://onlinelibrary.wiley.com/doi/10.1111/j.1540-6261.1952.tb01525.x)) proposed balancing **expected return** against **risk**, measured by variance. This is what modern portfolio theory relies on.

> A basic introduction to convex optimization will be useful here. See below section.

### 3.1 Two equivalent formulations

**Risk-constrained form**: Maximizing expected returns keeping risk bounded.
$$
\begin{aligned}
\max_w \quad & \mu^\top w \\
\text{s.t.} \quad
& w^\top \Sigma w \le \sigma^2_{\text{target}}, \\
& \mathbf{1}^\top w = 1.
\end{aligned}
$$

**Risk-penalized form**: Maximizing a weighted objective function.
$$
\begin{aligned}
\max_w \quad & \mu^\top w - \lambda w^\top \Sigma w \\
\text{s.t. } \quad & \mathbf{1}^\top w = 1.
\end{aligned}
$$

As $\sigma_{\text{target}}$ or $\lambda$ varies, these formulations trace out the same set of optimal portfolios. Note, the set of portfolios may be empty for some values of $\sigma_{\text{target}}$.



### 3.2 Evaluating and comparing portfolios

Once a portfolio $w$ is constructed, we need criteria to compare different portfolios.
In this project, we focus on three complementary metrics.

#### Risk–return tradeoff and the efficient frontier

Each portfolio corresponds to a point
$$
(\; w^\top \Sigma w,\; \mu^\top w \;)
$$
in (risk, return) space.

Varying the risk-aversion parameter $\lambda$ in the Markowitz formulation traces out
the **efficient frontier**: the set of portfolios that achieve the highest expected return
for a given level of risk.

#### Sharpe ratio

The motivation for the Sharpe Ratio (SR) is to focus on so-called "volatility-adjusted returns" as a summary metric. Formally,
$$
\text{SR}(w) = \frac{\mu^\top w}{\sqrt{w^\top \Sigma w}}.
$$

*Bonus Qn:* Considering its efficacy it would it optimal to choose the sharpe ratio as the maximized objective in our optimization tasks. But is that an easy problem to solve? What challenges might occur?  

#### (BONUS) Drawdown

If a *time series of realized returns* is available, we may additionally evaluate
**maximum drawdown**, which measures the largest peak-to-trough loss over time.

We don't use drawdown here but its an important concept that you can read about in the book.

### ⚠️ Word of caution
The analyses we do in this week are **static, in-sample** performance analysis, that is, we fix a predicted returns vector and covariance matrix, optimize the portfolio on that and study its expected performance.

For managing a real portfolio, one would need to test their portfolio on time series data, plot how their total wealth varies over:
- a **long horizon** (in that context our approach can be thought of as "locally greedy" as we optimize it on the basis of next tick's prediction)
- and possibly **rebalance** it on a fixed schedule (this would incur transaction costs further complicating the problem)

We ignore these for now deferring it for the next week, and focus on studying the theory and basic applications in this week


### Reference
**Main reference**. Chapter 7 from the portfolio optimization book.

**Performance metrics**. Section 6.3 from the book.

**Convex Optimization**. This only requires a basic understanding which can be gained from Stephen Boyd's book and the parallel course he taught at Stanford.
The [lecture slides](https://stanford.edu/~boyd/cvxbook/bv_cvxslides.pdf) cover the content in extreme detail with practical applications - see Ch 4 for intro to convex optimization problems.

**Completely unrelated :)**. A more theoretical introduction can be found in *Bertsekas, Convex Optimization Theory*. Boyd's book (though I haven't read it) extensively covers theory and algorithms from a practical optimization point of view while Bertsekas has a more rigorous flavour.

---

## 4. Convex Optimization

This section explains **how portfolio optimization fits into the general framework of convex optimization**. We proceed in four steps:

1. How convex optimization problems are framed
2. Where Markowitz fits (QP / SOCP)
3. Duality and the analytic solution of classical Markowitz
4. Why the analytic solution explains instability - and motivates robustness

A **convex optimization problem** has the general form
$$
\begin{aligned}
\min_x \quad & f(x) \\
\text{s.t.}\quad & x \in \mathcal{C},
\end{aligned}
$$
where:

* $f$ is a **convex function**, and
* $\mathcal{C}$ is a **convex set**.

#### Why convexity matters

Convexity guarantees:

* **no local minima** (every local minimum is global),
* **stable numerical solvers**,
* **duality theory** with strong guarantees.

These properties are what make large-scale portfolio optimization practical.


### 4.1 Convex sets, halfspaces, and polytopes

Some commonly studied convex sets are:
- A **halfspace** is a set of the form ${x : a^\top x \le b}$.
- An **affine set** is defined by linear equalities, e.g. ${x : A x = b}$.
- A **convex polytope** is the intersection of finitely many halfspaces and affine sets.

In portfolio optimization:

* $ \mathbf{1}^\top w = 1 $ is an affine constraint,
* $ w_i \ge 0 $ is an intersection of halfspaces,
* additional bounds $\ell \le w \le u$ preserve convexity.

Hence, the feasible portfolios form a **convex polytope** in $\mathbb{R}^n$.

### 4.2 Quadratic programs and SOCPs

A **quadratic program (QP)** has objective
$$
\min_x ; \tfrac12 x^\top Q x + c^\top x
$$
with linear constraints, where $Q \succeq 0$ (positive semi def).

The classical Markowitz objective $w^\top \Sigma w$ is exactly of this form.

Some equivalent formulations (e.g. volatility constraints)
$$
\sqrt{w^\top \Sigma w} \le \sigma
$$
can also be written as **second-order cone programs (SOCPs)**. This is a subclass of convex programs with efficient solvers, owing to various reasons (for eg. dual of a SOCP is an SOCP etc.)

## 5. Classical Markowitz as a convex program

We now write the **risk-adjusted return** formulation:
$$
\begin{aligned}
\max_w \quad & \mu^\top w - \lambda w^\top \Sigma w \
\text{s.t.}\quad & \mathbf{1}^\top w = 1.
\end{aligned}
\tag{M}
$$

This is a convex quadratic program because:

* the objective is concave (linear minus convex quadratic),
* maximizing a concave function is equivalent to minimizing a convex one
* the constraint is affine.

At this stage, **no long-only constraint** is imposed — this is intentional to keep the analysis simple.

---

## 6. (OPTIONAL) Duality and the analytic solution

This section analyzes problem $M$ using **Lagrangian duality**. Since the convexity of the objective and feasibility set have already been established earlier, we focus here on how duality exposes structure and yields an explicit solution.

### 6.0 Dual formulation

Consider a general convex program
$$
\min_{x \in \mathbb{R}^n} f(x)
\quad \text{s.t. } Ax = b,
$$
where $f$ is a [proper](https://en.wikipedia.org/wiki/Proper_convex_function) [closed convex](https://en.wikipedia.org/wiki/Closed_convex_function) function and the constraints are affine. This is called the **primal**.

> The lagrangian can be define in a more general setting as well, see the [slides](https://web.stanford.edu/~boyd/cvxbook/bv_cvxslides.pdf), page 155.

The **Lagrangian** associated with this problem is
$$
\mathcal{L}(x,\nu) = f(x) + \nu^\top (b - Ax),
\quad \nu \in \mathbb{R}^m.
$$

> The rigorous definitions of the above highlighted terms can be found in Bertsekas. For the interested readers, I have attached a report based on that [below](#65-other-duality-interpretations---further-reading), although you might be able to find satisfactory explanations online. 

For any fixed $\nu$, minimizing $\mathcal{L}(x,\nu)$ over $x$ produces a lower bound on the primal optimal value. Maximizing this bound over $\nu$ defines the **dual problem**
$$
\max_{\nu \in \mathbb{R}} \;\; \inf_{x} \mathcal{L}(x,\nu).
$$

For convex programs with affine constraints, [**strong duality**](https://people.eecs.berkeley.edu/~elghaoui/Teaching/EE227A/lecture8.pdf) holds under mild regularity conditions, and the primal and dual optimal values coincide. Moreover, at optimality the primal and dual variables satisfy the [**stationarity (KKT) condition**](https://www.stat.cmu.edu/~ryantibs/convexopt-F16/scribes/kkt-scribed.pdf)
$$
\nabla_x \mathcal{L}(x^*,\nu^*) = 0,
$$
which allows recovery of the primal solution from the dual variable.

> Strong Duality: roughly, it means that the optimal value for the dual problem equals that for the primal, and it enables proving some powerful results on the existence of optimal solutions to both.

> KKT Conditions: a concise representation of the necessary and sufficient conditions for existence of a pair of optimal solutions to the primal and dual problems.

### 6.1 Lagrangian for the Markowitz problem

We consider the Markowitz objective
$$
\max_{w} \;\; \mu^\top w - \lambda w^\top \Sigma w
\quad \text{s.t. } \mathbf{1}^\top w = 1,
$$
with $\lambda > 0$ and $\Sigma \succ 0$.

Introduce a scalar dual variable $\nu \in \mathbb{R}$ for the budget constraint. The Lagrangian is
$$
\mathcal{L}(w,\nu)
= \mu^\top w
- \lambda w^\top \Sigma w
+ \nu \bigl(1 - \mathbf{1}^\top w\bigr).
$$


### 6.2 Stationarity condition

At optimality,
$$
\nabla_w \mathcal{L}(w,\nu) = 0
\quad \Longrightarrow \quad
\mu - 2\lambda \Sigma w - \nu \mathbf{1} = 0.
$$

Solving for $w$,
$$
w^*
= \frac{1}{2\lambda}\,\Sigma^{-1}(\mu + \nu \mathbf{1}).
\tag{1}
$$

This expression reveals a key structural feature: the inverse covariance matrix $\Sigma^{-1}$ appears explicitly in the solution.


### 6.3 Solving for the dual variable

Impose the feasibility constraint $\mathbf{1}^\top w^* = 1$. Substituting (1),
$$
\frac{1}{2\lambda}\,
\mathbf{1}^\top \Sigma^{-1}(\mu + \nu \mathbf{1}) = 1.
$$

Solving for $\nu$ yields
$$
\nu^*
=
\frac{2\lambda - \mathbf{1}^\top \Sigma^{-1}\mu}
{\mathbf{1}^\top \Sigma^{-1}\mathbf{1}}.
$$

Substituting $\nu^*$ back into (1) gives the closed-form Markowitz solution.


### 6.4 Dual interpretation

The dual variable $\nu^*$ enforces the budget constraint and adjusts the effective returns before inversion by $\Sigma$. As a consequence, the portfolio weights depend on $\Sigma^{-1}\mu$ and $\Sigma^{-1}\mathbf{1}$, making the solution highly sensitive to estimation error in the covariance matrix. This observation motivates robust and regularized formulations, where uncertainty in $\mu$ or $\Sigma$ is handled directly at the level of the optimization problem.

### 6.5 Other duality interpretations - further reading
The lagrangian formulation is standard for these problems, but there are other frameworks for motivating the need for duality for example the MC/MC duality framework (geometrically motivates the dual problem) is discussed in Bertsekas. A brief technical report on it can be found [here](https://drive.google.com/file/d/14vXHzbnA7RnTs2IKFctn6bFI8CSoGIdr/view?usp=sharing), it is based on the first 4 chapters from Bertsekas.

## 7. Why the analytic solution is unstable

The closed-form Markowitz solution involves the inverse covariance matrix $\Sigma^{-1}$. This is the **primary source of instability**.

If $\Sigma$ has highly correlated assets or very small eigenvalues, then $\Sigma^{-1}$ has
very large eigenvalues. As a result, small errors in the estimated mean returns are magnified
by the optimization.

Formally,
$$
\|\delta \mu\|\ \text{small}
\quad\Longrightarrow\quad
\|\delta w^*\|\ \text{large}.
$$

Mean–variance optimization therefore **amplifies estimation noise** in the inputs.

### Geometric perspective

In portfolio weight space, the eigenvectors of $\Sigma$ define principal risk directions.
Directions corresponding to small eigenvalues are weakly penalized by the risk term,
allowing large changes in weights at little apparent cost.

This leads to:
- extreme portfolio weights,
- frequent sign flips,
- high turnover across time.

These effects are structural and unavoidable in the classical formulation.

---



## 8. Robust optimization: motivation and standard approach

The response to this instability is **not** to abandon optimization, but to explicitly
model uncertainty in the inputs.

Instead of assuming a single estimated mean $\hat\mu$, we assume the true mean lies in
an uncertainty set
$$
\mu \in \mathcal U
\;=\;
\{\hat\mu + \delta : \|\delta\|_\infty \le \varepsilon\}.
$$

We then solve the **robust Markowitz problem**. In two ways:
1. *Stochastic Optimization:* Optimizing with respect to the *expected* value of the objective function, assuming some probability distribution over $\mu, \Sigma$.
2. *Adversarial Optimization:* Optimizing for the *worst-case* inputs for $\mu, \Sigma$.

We focus on the worst-case formulation here.

$$
\max_w \;\min_{\mu \in \mathcal U}
\;\mu^\top w - \lambda w^\top \Sigma w.
$$

> The objective is concave: infimum of finite number of convex functions = negative of supremum of finite number of convex functions = negative of convex function
> Hence the maximization problem is a well-framed CP.

This satisfies strong duality and hence is tractable. See Ch 14.2 in the book for this.

## 8.1 Robust uncertainty and the resulting optimization problem

To make the optimization less sensitive to estimation error, we replace the point estimate
$\hat\mu$ by a simple uncertainty model. Assume the true mean return vector lies in a norm
ball around the estimate:
$$
\mu = \hat\mu + \delta,
\qquad
\|\delta\|_\infty \le \varepsilon.
$$

We then solve the robust objective
$$
\max_w \;\min_{\|\delta\|_\infty \le \varepsilon}
(\hat\mu + \delta)^\top w - \lambda w^\top \Sigma w.
$$

The inner minimization has an explicit solution. Since $\delta^\top w$ is linear in $\delta$,
the worst case occurs when each component of $\delta$ has opposite sign to $w$ and maximal
magnitude:
$$
\min_{\|\delta\|_\infty \le \varepsilon} \delta^\top w
=
-\varepsilon \sum_i |w_i|
=
-\varepsilon \|w\|_1.
$$

Substituting this back into the objective yields the equivalent single-level problem
$$
\max_w
\;\hat\mu^\top w
- \lambda w^\top \Sigma w
- \varepsilon \|w\|_1.
$$

The effect of robustness is therefore explicit: portfolios with large absolute weights are
penalized. Directions in which the optimizer would otherwise take extreme positions become
more costly.

This problem remains convex and can be solved using the same methods as the classical
Markowitz formulation. The only change is the additional $\ell_1$ term, which directly
controls the magnitude of the weights.

## 8.2 Bonus — Alternative: stochastic optimization

An alternative way to model uncertainty in the expected returns is to treat the estimation
error as **random** rather than adversarial.

Suppose the true mean return satisfies
$$
\mu = \hat\mu + \delta,
\qquad
\mathbb{E}[\delta] = 0,
\qquad
\mathrm{Cov}(\delta) = \tau^2 I,
$$
where $\delta$ represents estimation noise and $\tau^2$ controls its magnitude.

Instead of optimizing for the worst case, we optimize the **expected objective**:
$$
\max_w \;\mathbb{E}\big[(\hat\mu + \delta)^\top w - \lambda w^\top \Sigma w\big].
$$

Taking expectation,
$$
\mathbb{E}[(\hat\mu + \delta)^\top w]
=
\hat\mu^\top w,
$$
so the mean term is unchanged. However, the variance of the portfolio return increases due
to uncertainty in $\mu$:
$$
\mathrm{Var}(\delta^\top w)
=
\tau^2 \|w\|_2^2.
$$

A simple way to account for this additional risk is to penalize it directly, leading to
the modified objective
$$
\max_w
\;\hat\mu^\top w
- \lambda w^\top \Sigma w
- \gamma \|w\|_2^2,
$$
where $\gamma$ is proportional to $\tau^2$.

This formulation has a closed-form interpretation: the effective risk matrix becomes
$$
\Sigma_{\text{eff}} = \Sigma + \tfrac{\gamma}{\lambda} I.
$$

Thus, stochastic modeling of estimation error leads to **quadratic regularization** of the
portfolio weights. Compared to the worst-case robust formulation:
- $\ell_2$ regularization discourages large weights smoothly,
- ill-conditioned directions of $\Sigma$ are damped uniformly.


## 9. Why convexity matters again

The robust formulation:
- remains convex,
- admits strong duality,
- can be solved efficiently and reliably.

Without convexity, minimax formulations would be computationally intractable.

In fact the robust formulations are closely related to the "regularization" techniques used in standard regression models.

---

## 10. Summary

- The appearance of $\Sigma^{-1}$ explains the instability of classical Markowitz portfolios.
- Sensitivity is a mathematical consequence of optimization.
- Robust optimization modifies the objective to account for uncertainty.
- Regularization emerges from principled modeling, not heuristics.